In [8]:
from tensorflow.python.client import device_lib 
print(device_lib.list_local_devices())
import tensorflow as tf
print(tf.__version__)

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 13036984228523371193
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 1747976192
locality {
  bus_id: 1
  links {
  }
}
incarnation: 11229835717818532211
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6"
xla_global_id: 416903419
]
2.10.1


In [9]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [10]:
from keras.preprocessing.image import ImageDataGenerator

batch_size = 128
img_path_train = "datasets/food-101/food-101/images/"

train_datagen = ImageDataGenerator(rescale= 1.0/255, validation_split=0.2)


train_generator = train_datagen.flow_from_directory(img_path_train,
                                                    target_size=(24,24),
                                                    color_mode="rgb",
                                                    batch_size=batch_size,
                                                    class_mode='sparse',
                                                    shuffle=True,
                                                    subset='training')

validation_generator = train_datagen.flow_from_directory(img_path_train,
                                                    target_size=(24,24),
                                                    color_mode="rgb",
                                                    batch_size=batch_size,
                                                    class_mode='sparse',
                                                    shuffle=True,
                                                    subset='validation')

Found 80800 images belonging to 101 classes.
Found 20200 images belonging to 101 classes.


In [4]:
df = pd.read_csv('datasets/nutrition101 (1).csv')
protein_labels = list(df['protein'])

In [5]:
def regression_gen(flow_from_directory_gen, list_of_values):
    for x, y in flow_from_directory_gen:
        yield x, np.array([list_of_values[int(y_i)] for y_i in y])

In [6]:
from keras.layers import Dense, Input, Dropout, GlobalAveragePooling2D, Flatten, Conv2D, BatchNormalization, Activation, MaxPooling2D
from keras.models import Model, Sequential
from keras.optimizers import Adam


# Initialising the CNN
model = Sequential()

#Convolution 1
model.add(Conv2D(64,(3,3), padding='same', input_shape=(24, 24, 3)))
model.add(Activation('relu'))

# #Convolution 2
# model.add(Conv2D(64,(3,3), padding='same', input_shape=(128, 128, 3)))
# model.add(Activation('relu'))

# Flattening
model.add(Flatten())

# Fully connected layer 1st layer
model.add(Dense(256))
model.add(Activation('relu'))

# model.add(Dense(nb_classes))
model.add(Dense(1, activation='linear'))

print(model.summary())

opt = Adam(learning_rate=0.1)
# model.compile(optimizer=opt, loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
model.compile(optimizer=opt, loss="mean_absolute_percentage_error", metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 24, 24, 64)        1792      
                                                                 
 activation (Activation)     (None, 24, 24, 64)        0         
                                                                 
 flatten (Flatten)           (None, 36864)             0         
                                                                 
 dense (Dense)               (None, 256)               9437440   
                                                                 
 activation_1 (Activation)   (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                                 
Total params: 9,439,489
Trainable params: 9,439,489
Non-

In [7]:
# number of epochs to train the NN
epochs = 1

# checkpoint to save best model
from keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint("model_weights.h5", monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]
with tf.device('/gpu:0'):
    history = model.fit_generator(
        generator=regression_gen(train_generator, protein_labels),
        steps_per_epoch=train_generator.n//train_generator.batch_size,
        epochs=epochs,
        validation_data = regression_gen(validation_generator, protein_labels),
        validation_steps = validation_generator.n//validation_generator.batch_size,
        callbacks=callbacks_list
    )

C:\Users\Emeze\AppData\Local\Temp\ipykernel_3840\241174189.py:10: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


631/631 [==============================] - ETA: 0s - loss: 546269440.0000 - accuracy: 0.0297

KeyboardInterrupt: 

In [30]:
model.predict(train_generator.next()[0][0:1, :, :, :])

1/1 [==============================] - 0s 107ms/step


array([[0.00373659]], dtype=float32)